In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
import pandas as pd
from tqdm import tqdm

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model_name = "facebook/mbart-large-50"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained("./mbart-nepali-summarization").to(device)

def batch_summarize(texts, batch_size=4, max_length=150, min_length=50):
    """Batch summarize Nepali texts using mBART"""
    summaries = []
    
    # Set Nepali language code once
    tokenizer.src_lang = "ne_NP"
    forced_bos_token_id = tokenizer.lang_code_to_id["ne_NP"]
    
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        
        # Tokenize batch with proper padding
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(device)
        
        # Generate summaries
        generated_tokens = model.generate(
            **inputs,
            max_length=max_length,
            min_length=min_length,
            num_beams=4,
            early_stopping=True,
            forced_bos_token_id=forced_bos_token_id
        )
        
        # Decode batch
        batch_summaries = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True
        )
        
        summaries.extend(batch_summaries)
    
    return summaries

# Load test data (adjust column name as needed)
test_df = pd.read_csv("test_dataset.csv")
texts = test_df["text"].tolist()  # Assuming column named "text"

# Process in batches
print("Generating summaries...")
test_df["predicted_summary"] = batch_summarize(texts)

# Save results
output_path = "mbart_predictions_with_summaries.csv"
test_df.to_csv(output_path, index=False)
print(f"\nPredictions saved to {output_path}")

Generating summaries...


 26%|███████████                                | 23/89 [03:20<09:55,  9.02s/it]

In [35]:
from rouge import Rouge
import pandas as pd

# Load your predictions file
predictions_df = pd.read_csv("mbart_predictions_with_summaries.csv")

# Initialize ROUGE evaluator
rouge = Rouge()

# Calculate ROUGE scores
scores = rouge.get_scores(
    predictions_df["predicted_summary"].tolist(),  # Generated summaries
    predictions_df["summary"].tolist(),           # Reference summaries (gold standard)
    avg=True
)

# Print formatted results
print("ROUGE Scores:")
print(f"ROUGE-1: f={scores['rouge-1']['f']:.4f}, p={scores['rouge-1']['p']:.4f}, r={scores['rouge-1']['r']:.4f}")
print(f"ROUGE-2: f={scores['rouge-2']['f']:.4f}, p={scores['rouge-2']['p']:.4f}, r={scores['rouge-2']['r']:.4f}")
print(f"ROUGE-L: f={scores['rouge-l']['f']:.4f}, p={scores['rouge-l']['p']:.4f}, r={scores['rouge-l']['r']:.4f}")

ROUGE Scores:
ROUGE-1: f=0.1893, p=0.1700, r=0.2266
ROUGE-2: f=0.0642, p=0.0558, r=0.0806
ROUGE-L: f=0.1685, p=0.1509, r=0.2021
